In [2]:
from datasets import load_dataset

ds = load_dataset("hackercupai/hackercup")
ds

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    sample: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 10
    })
    full: Dataset({
        features: ['name', 'year', 'round', 'statement', 'input', 'solution', 'code', 'output', 'sample_input', 'sample_output', 'images'],
        num_rows: 284
    })
})

In [6]:
ds['sample']['name']

['meta_game',
 'tower_rush',
 'ready_go_part_2',
 'wiki_race',
 'ready_go_part_1',
 'dim_sum_delivery',
 'cheeseburger_corollary_ch1',
 'two_apples_a_day',
 'cheeseburger_corollary_ch2',
 'road_to_nutella']

In [92]:
import threading
import queue
import traceback
import sys
import io
import contextlib
from contextlib import contextmanager
from unittest.mock import patch

class TimeoutException(Exception):
    pass

class CodeExecutionThread(threading.Thread):
    def __init__(self, code_str, test_input):
        threading.Thread.__init__(self)
        self.code_str = code_str
        # Improve input handling
        self.test_input_lines = []
        if test_input:
            # Split input and remove empty lines
            self.test_input_lines = [
                line for line in test_input.strip().split('\n')
                if line and not line.isspace()
            ]
        self.original_input_count = len(self.test_input_lines)  # Keep track of original input count
        self.input_position = 0  # Track current input position
        self.queue = queue.Queue()
        self.output = io.StringIO()
        
    def mock_input(self):
        if not self.test_input_lines:
            error_msg = (
                f"Input exhausted after reading {self.input_position} lines. "
                f"Original input had {self.original_input_count} lines. "
                "Your code is trying to read more input than provided."
            )
            raise ValueError(error_msg)
        
        self.input_position += 1
        return self.test_input_lines.pop(0)
        
    def run(self):
        try:
            with patch('builtins.input', self.mock_input), contextlib.redirect_stdout(self.output):
                code_obj = compile(self.code_str, '<string>', 'exec')
                local_scope = {'__name__': '__main__'}
                exec(code_obj, local_scope)
            
            result = self.output.getvalue()
            # Also report if there's unused input
            if self.test_input_lines:
                unused_lines = len(self.test_input_lines)
                result += f"\nWarning: {unused_lines} lines of input were not used."
            
            self.queue.put(("success", result))
                
        except Exception as e:
            exc_type, exc_value, exc_traceback = sys.exc_info()
            tb = traceback.extract_tb(exc_traceback)
            line_no = tb[-1].lineno
            code_lines = self.code_str.split('\n')
            error_line = code_lines[line_no - 1] if line_no <= len(code_lines) else "Unknown"
            
            # Enhanced error message
            error = (
                f"Error on line {line_no}: {error_line.strip()}\n"
                f"Exception: {exc_type.__name__}: {str(exc_value)}\n"
                f"Input position when error occurred: {self.input_position}\n"
                f"Remaining unused input lines: {len(self.test_input_lines)}"
            )
            self.queue.put(("error", error))

def run_code_with_timeout(code_str, test_input, timeout=20):
    """
    Run code with timeout and improved input handling.
    
    Args:
        code_str (str): The Python code to execute
        test_input (str): Input data, with lines separated by newlines
        timeout (int): Maximum execution time in seconds
    
    Returns:
        tuple: (result, error_message)
    """
    
    execution_thread = CodeExecutionThread(code_str, test_input)
    execution_thread.start()
    execution_thread.join(timeout)
    
    if execution_thread.is_alive():
        execution_thread = None
        return None, "The code execution timed out."
    
    try:
        status, result = execution_thread.queue.get_nowait()
        if status == "success":
            return result, None
        else:
            return None, result
    except queue.Empty:
        return None, "Execution failed with no output"

In [93]:
extracted_code = "import heapq\n\n\ndef generate_hops(R, C, grid):\n    hops = []\n    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]\n    for i in range(R):\n        for j in range(C):\n            for di, dj in directions:\n                ni, nj = i + di, j + dj\n                if 0 <= ni < R and 0 <= nj < C and grid[i][j] != grid[ni][nj]:\n                    score = max(abs(i - ni), abs(j - nj))\n                    hops.append(score)\n    return hops\n\n\ndef kth_smallest_score(hops, K):\n    min_heap = []\n    for score in hops:\n        heapq.heappush(min_heap, score)\n    for _ in range(K - 1):\n        heapq.heappop(min_heap)\n    return min_heap[0]\n\n\ndef main():\n    T = int(input())\n    for case in range(1, T + 1):\n        R, C, K = map(int, input().split())\n        grid = [list(map(int, input().split())) for _ in range(R)]\n        hops = generate_hops(R, C, grid)\n        result = kth_smallest_score(hops, K)\n        print(f'Case #{case}: {result}')\n\nif __name__ == '__main__':\n    main()"

inputs = """
6
121 121 11
0 100 2
0 132 1
121 132 1
121 131 1
22322 22322 1
"""
output = """
Case #1: 1
Case #2: 4
Case #3: 10
Case #4: 1
Case #5: 1
Case #6: 0
"""

run_code_with_timeout(extracted_code,inputs)

(None,
 'Error on line 37: Unknown\nException: ValueError: Input exhausted after reading 7 lines. Original input had 7 lines. Your code is trying to read more input than provided.\nInput position when error occurred: 7\nRemaining unused input lines: 0')